In [1]:
"""
Using a Poisson Filtered Mesh, use the cgal library to 
extract just the soma
"""


'\nUsing a Poisson Filtered Mesh, use the cgal library to \nextract just the soma\n'

In [4]:
import cgal_Segmentation_Module as csm
from whole_neuron_classifier_datajoint_adapted import extract_branches_whole_neuron
import time
import trimesh
import numpy as np
import datajoint as dj

In [3]:
dir(csm)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'cgal_demo',
 'cgal_segmentation']

In [5]:
#import the mesh
new_mesh = trimesh.load_mesh("96631955273149705_Poisson_mesh_largest_mesh.off")

(128586, 3)

In [12]:
segment_id = 96631955273149705
faces = np.array(new_mesh.faces)
verts = np.array(new_mesh.vertices)
#run the whole algorithm on the neuron to test
verts_labels, faces_labels = extract_branches_whole_neuron(import_Off_Flag=False,segment_id=segment_id,vertices=verts,
                     triangles=faces,pymeshfix_Flag=False,
                     import_CGAL_Flag=False,
                     return_Only_Labels=True,
                     clusters=3,
                     smoothness=0.2)

1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.0004878044128417969
2) Staring: Generating CGAL segmentation for neuron
Done writing OFF file

Starting CGAL segmentation
Right before cgal segmentation, clusters = 3, smoothness = 0.2
1
Finished CGAL segmentation algorithm: 27.641087770462036
2) Finished: Generating CGAL segmentation for neuron: 29.851298809051514
3) Staring: Generating Graph Structure and Identifying Soma
soma_index = 1
3) Finished: Generating Graph Structure and Identifying Soma: 0.04589676856994629
5) Staring: Finding Apical Index
Soma Index = 1
Soma Connections = [22, 12, 25, 7, 0, 10, 16, 13, 26]
soma_80_percent = 813370.42
Debugging the axon filter
[(22, 812200.0), (12, 816874.5), (25, 823553.8), (7, 833579.8), (0, 809054.0), (10, 822507.6), (16, 821722.7), (13, 830693.4), (26, 811477.4)]
possible_Axons_filter_1 = [22, 0, 26]
possible_Axons_filter_2 = []
possible_Apical = None
5

In [15]:
from collections import Counter
Counter(faces_labels)

Counter({10.0: 113526, 5.0: 15060})

In [20]:
soma_faces = np.where(faces_labels == 5.0)[0]

In [24]:
soma_mesh = new_mesh.submesh([soma_faces],append=True)
soma_mesh.export(str(segment_id) + "_soma.off")
print("help")

help


In [28]:
# calculate the soma center by averaging all of the vertices
soma_center = soma_mesh.vertices.mean(axis=0).astype("float")
soma_center

TrackedArray([ 968842.04038464,  817070.6912408 , 1055428.45468915])

# Testiing the meshlabserver script

In [ ]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    print('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command)
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result

In [ ]:
def meshlab_fix_manifold(key,folder="pymesh_NEURONS"):
    
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

def meshlab_fix_manifold_path(path_and_filename,segment_id=-1):
    #fix the path if it comes with the extension
    if path_and_filename[-4:] == ".off":
        path_and_filename = path_and_filename[-4:]
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    #print("input_mesh = " + str(input_mesh))
    #print("output_mesh = " + str(output_mesh))
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

def meshlab_fix_manifold_path_specific_mls(path_and_filename,segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if path_and_filename[-4:] == ".off":
        path_and_filename = path_and_filename[-4:]
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    #print("input_mesh = " + str(input_mesh))
    #print("output_mesh = " + str(output_mesh))
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh


In [ ]:
#Example of how to run it using the commands

#output_mesh = meshlab_fix_manifold_path(path_and_filename,key["segment_id"])
meshlab_script = str(pathlib.Path.cwd()) + "/" + "pymesh_fix_substitute.mls"
output_mesh = meshlab_fix_manifold_path_specific_mls(path_and_filename,key["segment_id"],meshlab_script)


In [ ]:
#what command would look like: 
meshlabserver -i /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/96631955273149705_platinum_neuron.off -o /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/96631955273149705_platinum_neuron_auto_mls.off -s /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/new_screened_poisson_filter.mls

In [ ]:
meshlabserver -i /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/96631955273149705_platinum_neuron.off -o /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/96631955273149705_platinum_neuron_auto_mls.off -s /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/poisson_vp2.mls

# 96631955273149705_soma.off

In [ ]:
meshlabserver -i /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/96631955273149705_soma.off -o /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/96631955273149705_soma_auto_mls.off -s /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/poisson_vp2.mls